In [1]:
import pandas as pd
import random
from sqlalchemy import create_engine
import numpy as np

1. On charge harge les fichiers sources

In [2]:
# Charger les fichiers sources
file1 = 'data\\ObesityDataSet_raw_and_data_sinthetic.csv'
file2 = 'data\\lifestyle_sustainability_data.csv'
file3 = 'data\\Sleep_health_and_lifestyle_dataset.csv'
file4 = 'data\\DietSurvey_toJson.json'

# Charger les fichiers dans des DataFrames
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)
df4 = pd.read_json(file4)

# Ajouter des IDs uniques pour chaque fichier
df1['id_person'] = range(1, len(df1) + 1)
df2['id_person'] = range(len(df1) + 1, len(df1) + len(df2) + 1)
df3['id_person'] = range(len(df1) + len(df2) + 1, len(df1) + len(df2) + len(df3) + 1)
df4['id_person'] = range(len(df1) + len(df2) + len(df3) + 1, len(df1) + len(df2) + len(df3) + len(df4) + 1)


2. On code les fonctions dont on aura besoin par la suite

In [3]:
def assign_diet_type(fcvc):
    if fcvc >= 2.6:
        # Si FCVC >= 2.6, attribuer "Vegetarian" ou "Vegan" avec probabilité pour "Vegan" plus faible
        return np.random.choice(['Vegetarian', 'Vegan'], p=[0.8, 0.2])
    elif fcvc >= 2:
        # Si FCVC >= 2 et < 2.6, attribuer "Mostly Plant-Based"
        return 'Mostly Plant-Based'
    elif fcvc <= 1:
        # Si FCVC <= 1, attribuer "Mostly Animal-Based"
        return 'Mostly Animal-Based'
    else:
        # Sinon, attribuer "Balanced"
        return 'Balanced'

In [4]:
# Traitement pour le fichier 1 (ObesityDataSet_raw_and_data_sinthetic.csv)
df1['diet_type'] = df1['FCVC'].apply(assign_diet_type)

# Traitement pour le fichier 2 (lifestyle_sustainability_data.csv)
# Ici, diet_type est simplement la colonne 'DietType'
df2['diet_type'] = df2['DietType']

# Traitement pour le fichier 3 (DietSurvey_toJson.json)
# Extraire la réponse avant les parenthèses dans la question "What would best describe your diet:"
# Si la réponse contient une parenthèse, on prend la partie avant la parenthèse ; sinon, on garde la réponse telle quelle
df4['diet_type'] = df4['What would best describe your diet:'].apply(lambda x: x.split('(')[0].strip() if '(' in x else x.strip())


3. On regarde les différentes valeurs obtenues

In [5]:
# Afficher les différentes valeurs uniques possibles pour file2 et file3
file1_unique_values = df1['diet_type'].unique()
file2_unique_values = df2['diet_type'].unique()
file4_unique_values = df4['diet_type'].unique()

# Combiner toutes les valeurs uniques dans un seul ensemble pour éviter les doublons
unique_diet_values = set(file1_unique_values).union(file2_unique_values).union(file4_unique_values)

# Afficher les valeurs uniques combinées
print("Valeurs uniques combinées pour diet_type :", unique_diet_values)

Valeurs uniques combinées pour diet_type : {'Pollotarian', 'Mostly Plant-Based', 'Pollo-pescetarian', 'Vegan', 'Vegetarian', 'Pescatarian', 'Non-Vegetarian', 'Mostly Animal-Based', 'Balanced', 'Eggetarian'}


4. On fabrique le Dataframe correspondant

In [6]:
# Créer un DataFrame avec les valeurs uniques et un ID pour chaque régime
df_diet = pd.DataFrame(unique_diet_values, columns=['diet_type'])
df_diet['id_diet'] = df_diet.index + 1  # Assigner un id_diet unique à chaque régime

# Réorganiser les colonnes pour que 'id_diet' soit la première colonne
df_diet = df_diet[['id_diet', 'diet_type']]

# Afficher le DataFrame créé
print("\nDataFrame des régimes alimentaires :")
print(df_diet)



DataFrame des régimes alimentaires :
   id_diet            diet_type
0        1          Pollotarian
1        2   Mostly Plant-Based
2        3    Pollo-pescetarian
3        4                Vegan
4        5           Vegetarian
5        6          Pescatarian
6        7       Non-Vegetarian
7        8  Mostly Animal-Based
8        9             Balanced
9       10           Eggetarian


from db_utils import get_db_engine

# Obtenir le moteur de base de données
engine = get_db_engine()

df_diet.to_sql('Diet', engine, if_exists='append', index=False)

print("Les données ont été insérées avec succès dans la table 'Diet'.")


In [7]:
# Pour associer les diet_types aux id_diet dans chaque DataFrame
# On utilise pd.merge() pour lier les ID du régime alimentaire en fonction des diet_types

# Associer l'id_diet pour df1
df1 = pd.merge(df1, df_diet, how='left', left_on='diet_type', right_on='diet_type')

# Associer l'id_diet pour df2
df2 = pd.merge(df2, df_diet, how='left', left_on='diet_type', right_on='diet_type')

# Associer l'id_diet pour df4
df4 = pd.merge(df4, df_diet, how='left', left_on='diet_type', right_on='diet_type')

# Pour df3, comme il n'y a pas de 'diet_type', on attribue l'ID 11
df3['id_diet'] = 11

# Créer un DataFrame final avec les id_person et id_diet
id_and_diet = pd.concat([df1[['id_person', 'id_diet']],
                         df2[['id_person', 'id_diet']],
                         df3[['id_person', 'id_diet']],
                         df4[['id_person', 'id_diet']]])

# Réinitialiser l'index
id_and_diet.reset_index(drop=True, inplace=True)

# Afficher le DataFrame final
print("\nDataFrame des IDs de personnes et des régimes alimentaires :")
id_and_diet



DataFrame des IDs de personnes et des régimes alimentaires :


,id_person,id_diet
0,1,2
1,2,5
2,3,2
3,4,5
4,5,2
...,...,...
3057,3058,1
3058,3059,7
3059,3060,7
3060,3061,1
